In [1]:
from typing import Tuple

In [2]:
f = open('day11_input.txt')
lines = f.readlines()
f.close()
data = [(line[0:-1]) for line in lines]

In [3]:
print(f'There are {len(data)} rows of seatings. Each row has {len(data[0])} seats. The first few are as following:')
data[0:10]

There are 99 rows of seatings. Each row has 92 seats. The first few are as following:


['LLLLLLLLLL.LLLLLL.LLLLLLLLLLLL.LL.LLLL.LLLLL.LLLLLLL.LLLLLL.LLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLL',
 'LLLLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLLL.LLLLL.LLLLLLL.LLLLLLLLLLLLL.LLLL.L.LLLLLLLLLL.LLLLLLL',
 'LLLLLLLLLL.LLLLLL.LLLLLL.LLLLL.LLLLLLLLLLLLLLLLLLLLL.LLLLLL..LLLLL.LLLLLL.LLLLLL.LLLLLLLLLLL',
 'LLLLLLLLLL.LLLLLLLLLLLLL.LL.LL.LLLLL.L.LLLLL.LLLLLLL.LLLLLL.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLLL',
 'LLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLLL.LLLLLLL.LLL.LL.LLLLLL.LLLLLL.LLLLLLLLLLLLLLLLLL',
 'LLLLLLLLLLLLLLLLL.LLLLLL.LLLLL.LLLLLLL.LLL.L.LLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLL.LL',
 'LLLLLLLLLLLLLLLLL.LLLLLLLLLL.LLLLLLLLL.LLLLL.LLLLLLL.LLLL.L.LLLLLLLLLLLLL.LLLLLLLLLLLLLLLLLL',
 '.L.LL...LLLL.......L....L.LLLLLL.......LL....LL...L..L.LLL...LLL..L.L.L.L..L...............L',
 'LLLLLLLLLL.LL.LLL.LLLLLL.LLLLL.LLLLLLL.LLLLL.LLLLLLLLLLLLLLLLLLLLLLLLLLLL.LLLLLL.LLLLLLLLLLL',
 'LLLLLLLLLL.LLLLLL.LLLLLLL.LLLLLLLLLLLL.LLLLL.LLLL.LLLLLLLLL.LLLLLL.LLLLLL.LLLLLL.LLLLLLLLLLL']

Note: The seat layout fits on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). 

# Part 1:

This question is similar to the famous "Conway's Game of Life" (https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life).

Each seat will be taken (or vacant) based on the on the number of occupied seats adjacent to it (one of the eight positions immediately up, down, left, right, or diagonal from the seat).

The following rules are applied to every seat **simultaneously**:

- If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change.

For part 1, we want to know how many seats end up occupied at the end (until no seats change state).

Usually there are two way to do it "simultaneously": 1. to make a new copy of the previous seats every round, and make changes to the copy, not the previous seats; 2. to change the previous seats, but in a way that it still holds the original information.

Here I'm using the first approach, making a copy.

In [4]:
def check_rules(seatings: list) -> Tuple[bool, list]:
    """
    This function will check each seat in the seating list and make changes according to the two rules, and return True if 
    any seat has changed state, False if none seat has changed state, along with the new seats states.
    """
    n_row = len(seatings)
    n_col = len(seatings[0])
    # a new copy is made every time so that the "original" seats won't be modified:
    new_seatings = [['.']*n_col for _ in range(n_row)]
    # a flag to mark if any seat has changed state
    changed = False
    
    for i in range(0, n_row):
        for j in range(0, n_col):
            adjacents = [[i-1, j-1], [i-1, j], [i-1, j+1], [i, j-1], [i, j+1], [i+1, j-1], [i+1, j], [i+1, j+1]]
            # to check rule 1: 
            # If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.        
            if seatings[i][j] == 'L':
                all_empty = True
                for m, n in adjacents:
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            all_empty = False
                            break
                if all_empty:
                    new_seatings[i][j] = '#'
                    changed = True
                else:
                    new_seatings[i][j] = 'L'
            # to check rule 2: 
            # If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
            elif seatings[i][j] == '#':
                ct = 0
                for m, n in adjacents:
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                if ct >=4:
                    new_seatings[i][j] = 'L'
                    changed = True
                else:
                    new_seatings[i][j] = '#'

    return changed, new_seatings

With this function, we just need to let it run and stop when the returned boolen flag is False, which means no seat has changed state:

In [5]:
seatings = data
changed = True
i = 0
while changed:
    changed, seatings = check_rules(seatings)
    i += 1
print(f'{i} rounds passed.')

87 rounds passed.


To count occupied seats:

In [6]:
ct = 0
for i in range(len(seatings)):
    for j in range(len(seatings[0])):
        ct += int(seatings[i][j] == '#')
print(f'{ct} seats are accupied.')

2359 seats are accupied.


# Part 2

OK, new rules:

People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

It mean we can't use 

adjacents = [[i-1, j-1], [i-1, j], [i-1, j+1], [i, j-1], [i, j+1], [i+1, j-1], [i+1, j], [i+1, j+1]]

anymore.

Oh well, due to time limitation, here is my updated check_rules function v2 (brutal force version, not pretty... will need to simplify in the future):

In [7]:
def check_rules_v2(seatings: list) -> tuple:
    n_row = len(seatings)
    n_col = len(seatings[0])
    # a new copy is made every time so that the "original" seats won't be modified:
    new_seatings = [['.']*n_col for _ in range(n_row)]
    # a flag to mark if any seat has changed state
    changed = False
    
    for i in range(0, n_row):
        for j in range(0, n_col):
            # to check rule 1: 
            if seatings[i][j] == 'L':
                all_empty = True
                # direction 1 (some of the directions can be combined, of course)
                for x in range(1,100):
                    m, n = i-x, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d2
                for x in range(1,100):
                    m, n = i-x, j
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d3
                for x in range(1,100):
                    m, n = i-x, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d4
                for x in range(1,100):
                    m, n = i, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d5
                for x in range(1,100):
                    m, n = i, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d6
                for x in range(1,100):
                    m, n = i+x, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d7
                for x in range(1,100):
                    m, n = i+x, j
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                # d8
                for x in range(1,100):
                    m, n = i+x, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == 'L':
                            break
                        elif seatings[m][n] == '#':
                            all_empty = False
                            break
                
                if all_empty:
                    new_seatings[i][j] = '#'
                    changed = True
                else:
                    new_seatings[i][j] = 'L'
            
            # to check rule 2: 
            elif seatings[i][j] == '#':
                ct = 0
                # d1
                for x in range(1,100):
                    m, n = i-x, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d2
                for x in range(1,100):
                    m, n = i-x, j
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d3
                for x in range(1,100):
                    m, n = i-x, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d4
                for x in range(1,100):
                    m, n = i, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d5
                for x in range(1,100):
                    m, n = i, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d6
                for x in range(1,100):
                    m, n = i+x, j-x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d7
                for x in range(1,100):
                    m, n = i+x, j
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break
                # d8
                for x in range(1,100):
                    m, n = i+x, j+x
                    if (0 <= m < n_row) and (0 <= n < n_col):
                        if seatings[m][n] == '#':
                            ct += 1
                            break
                        elif seatings[m][n] == 'L':
                            break

                if ct >=5:
                    new_seatings[i][j] = 'L'
                    changed = True
                else:
                    new_seatings[i][j] = '#'

    return changed, new_seatings

In [8]:
seatings = data
changed = True
i = 0
while changed:
    changed, seatings = check_rules_v2(seatings)
    i += 1
print(f'{i} rounds passed.')

87 rounds passed.


In [9]:
ct = 0
for i in range(len(seatings)):
    for j in range(len(seatings[0])):
        ct += int(seatings[i][j] == '#')
print(f'{ct} seats are accupied.')

2131 seats are accupied.
